# Stock Sentiment Analysis from Headlines

More and more hedge funds and independent traders are utilizing data science to process the wealth of information available from news headlines in the quest for profit. In this project, I will generate investment insights by applying sentiment analysis on financial news headlines, webscraped from FINVIZ.com. Repsecting data science ethics with regard to webscraping, I have downloaded various HTML files for two large firms: Facebook & Tesla.

Through conducting sentiment analysis, we can examine the emotion behind the headlines and predict whether the market feels good or bad about a stock. Then, we can make educated guesses on how certain stocks will perform and trade accordingly. Below, we import these files:

In [3]:
# import libraries
from bs4 import BeautifulSoup
import os

html_tables = {}

# for every data set in os dataset folder
for table_name in os.listdir('datasets'):
    # filepath
    table_path = f'datasets/{table_name}'

    # open as read-only, read into 'html'
    table_file = open(table_path, 'r')
    html = BeautifulSoup(table_file)

    # add news-table to 'html_tables' dictionary
    html_table = html.find(id = 'news-table')
    html_tables[table_name] = html_table



We've now obtained the table which contains all the headlines from each stock's HTML file. Before we move any further, we must investigate the structure in the data table. Let's read a single day of headlines for Tesla. 

In [ ]:
tsla = html_tables['tsla_22sep.html']

# store all table rows with <tr> tags
tsla_tr = tsla.findAll('tr')

for i, table_row in enumerate(tsla_tr) :

    # store <a> elements in link_text
    link_text = table_row.a.get_text()
    # store <td> elements in data_text
    data_text = table_row.a.get_text()

    print(f'File number {i+1}:'); print(link_text); print(data_text)